In [19]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
from math import ceil, floor
sns.set_style('darkgrid')

In [20]:
TAB_SIZE = 4
EPS = 1e-9

In [21]:
def AVAR(q, w, alpha):
    eval = [(w[k], q[k])  for k in q.keys()]
    res = 0
    a = alpha
    for wk, qk in sorted(eval, reverse=True):
        if np.isclose(alpha, 0, atol = EPS):
            break
        if alpha >= qk:
            res += wk*qk
            alpha -= qk
        else:
            res += wk*alpha
            alpha = 0
    return res/a

def add_qw(qw1, qw2):
    # bad error for now
    Q_sum = {}
    w_sum = {}
    for x1, q1 in qw1[0].items():
        for x2, q2 in qw2[0].items():
            qs = q1*q2
            temp = qw1[1][x1] + qw2[1][x2]
            if isinstance(x1, int) or isinstance(x1, np.int32):
                x1 = (x1,)
            if isinstance(x2, int) or isinstance(x2, np.int32):
                x2 = (x2,)
            xs = x1 + x2
            Q_sum[xs] = qs 
            w_sum[xs] = temp
    return (Q_sum, w_sum)

def AVAR_of_sum(list_qw, alpha):
    return AVAR(*reduce(add_qw, list_qw), alpha)

def sum_of_AVAR(list_qw, alpha):
    return sum([AVAR(*qw, alpha) for qw in list_qw])

In [22]:
class Node:
    def __init__(self, id, model, parent = None, Q = {}, cost = {}):
        self.id = id
        self.parent = parent
        self.Q = Q
        self.cost = cost
        self.w = {}
        self.children = []
        self.policy = {}
        self.terminal = True
        self.model = model
        self.name = str(id)
        self.one_step = None
    
    def add_child(self, node):
        self.children.append(node)
        node.parent = self
        self.terminal = False
        
    def get_w(self):
        if self.w:
            return self.w
        if self.terminal:
            for x in self.model.X:
                self.w[x] = self.cost[x]
            return self.w
        self.calc_policy()
        return self.w    

    def calc_policy(self):
        """
            Calculate the optimal policy and value for the maximal subtree rooted here
        """
        if self.terminal:
            raise Error("calc_policy ran on terminal nodes!")
        def net_one_step(x, u):
            res = self.cost[u][x] + self.one_step([(child.Q[u][x], child.get_w()) for child in self.children])
            return res
        self.policy = {x: min(self.model.U, key=lambda u: net_one_step(x, u)) for x in self.model.X}
        self.w = {x: net_one_step(x, self.policy[x]) for x in self.model.X}
    
    def print_tree(self, level = 0):
        print(" " * TAB_SIZE * level + self.name)
        for child in self.children:
            child.print_tree(level+1)

In [23]:
class Model:
    def __init__(self, lo, hi, U, alpha):
        """
            State space X = [lo, hi] of interval size = 1
            Action space U
            VaR calculation alpha
            Assume that 0 is root node
        """
        self.X = range(lo, hi + 1)
        self.lo = lo
        self.hi = hi
        self.U = U
        self.alpha = alpha
        self.nodes = [Node(0, self)]
        self.root = self.nodes[0]
        self.construct_graph()
        self.construct_risks()
         
    def construct_graph(self):
        raise NotImplementedError("construct_graph has not been properly implemented!")
    
    def construct_risks(self):
        raise NotImplementedError("construct_risks has not been properly implemented!")
        
    
    def bound(self, q):
        q_res = {x : 0 for x in self.X}
        for k, qk in q.items():
            q_res[max(self.lo, min(k, self.hi))] += qk

        return q_res
    
    def draw_edge(self, parent_i, child_i):
        if max(parent_i, child_i) >= len(self.nodes):
            # add nodes appropriately
            self.nodes += [Node(i, self) for i in range(len(self.nodes), max(parent_i, child_i) + 1)]
        self.nodes[parent_i].add_child(self.nodes[child_i])

## R & D Model

In [27]:
class RDModel(Model):
    def __init__ (self, lo, hi, U, alpha, T, investment_cost = 1):
        """
            q0(x, u), q1(x, u)
            c0(x, u), c1(x)
        """
        self.T = T
        self.n = 2 * T + 2
        self.investment_cost = investment_cost
        super().__init__(lo, hi, U, alpha)

    def construct_graph(self):
        """
            customize graph structure here
        """
        for i in range(self.T):
            self.draw_edge(2*i, 2*i + 1)
            self.draw_edge(2*i, 2*i + 2)
        self.draw_edge(2*self.T, 2*self.T + 1)
    
    def construct_risks(self):
        """
            set Q, c, and one_step for each node
        """
        def q0(x, u, t):
            if u == 0:
                return {x-2: 0.2, x-1: 0.2, x: 0.2, x+1: 0.2, x+2: 0.2}
            # u == 1
            return {x+1: 0.4, x+2: 0.2, x+3: 0.4}

        def q1(x, u, t):
            if u == 0:
                return {x-1: 0.6, x: 0.2, x+1: 0.2}
            # u == 1
            return {x-1: 0.2, x: 0.4, x+1: 0.4}

        def c0(x, u, t):
            if u == 0:
                return 0
            return self.investment_cost

        def c1(x, u, t):
            # x = state
            # a = action of this node
            return np.exp(-x/20)
        
        for i in range(self.n):
            self.nodes[i].t = ceil(float(self.nodes[i].id)/2)
        for node in self.nodes:
            node.t = ceil(float(node.id)/2)
            if node.terminal:
                node.cost = {x : c1(x, None, node.t) for x in self.X}
                node.Q = {u : {x : self.bound(q1(x, u, node.t)) for x in self.X} for u in self.U}
            else:
                node.cost = {u : {x : c0(x, u, node.t) for x in self.X} for u in self.U}
                node.Q = {u : {x : self.bound(q0(x, u, node.t)) for x in self.X} for u in self.U}
            #!customize one step here
            node.one_step = lambda list_qw : sum_of_AVAR(list_qw, self.alpha)
        
    # customized functions for this particular model
    def policy_change(self, policy):
        res = self.lo
        for k, v in policy.items():
            if v == 1:
                res = k
        return res

In [ ]:
models = []
LO, HI = -100, 100
T_range = range(10, 21, 10)
investment_costs = range(1, 5, 2)
alphas = [0.1, 0.2, 0.3, 0.4, 0.5]

### Plot against investment_costs, fixed alpha = 0.3

In [ ]:
"""
    Plot against investment_costs
"""
frames = []
for investment_cost in investment_costs:
    frame = []
    for T in T_range:
        model = RDModel(LO, HI, [0, 1], 0.3, T, investment_cost)
        model.root.calc_policy()
        x = [node.t for node in model.nodes if node.policy]
        y = [model.policy_change(node.policy) for node in model.nodes if node.policy]
        frame.append(go.Scatter(x = x, y = y, mode = 'markers', name=f'T = {T}'))
    frames.append(go.Frame(data = frame, name = f'Cost {investment_cost}'))

In [ ]:
# Create the figure with the frames and the slider
fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title='Changes by Time with Cost Slider',
        xaxis=dict(title='Time'),
        yaxis=dict(title='Value of Change'),
        yaxis_range=[-50, 60],
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True, mode="immediate")]),
                     dict(label="Pause",
                          method="animate",
                          args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")])]
        )]
    ),
    frames=frames
)

# Configure the sliders
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_layout(
    yaxis=dict(dtick=10),
    xaxis=dict(dtick=10),
    sliders=[{
        'steps': [{'args': [[f.name], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate'}],
                   'label': f'{f.name}',
                   'method': 'animate'} for f in frames],
        'transition': {'duration': 300},
    }]
)
fig.show()

### Plot against alphas, fixed investment_cost = 5

In [ ]:
"""
    Plot against alphas
"""
frames = []
for alpha in alphas:
    frame = []
    for T in T_range:
        model = RDModel(LO, HI, [0, 1], alpha, T, 5)
        model.root.calc_policy()
        x = [node.t for node in model.nodes if node.policy]
        y = [model.policy_change(node.policy) for node in model.nodes if node.policy]
        frame.append(go.Scatter(x = x, y = y, mode = 'markers', name=f'T = {T}'))
    frames.append(go.Frame(data = frame, name = f'Alpha {alpha}'))

In [ ]:
# Create the figura with the frames and the slider
fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title='Changes by Time with Alpha Slider and cost = 5',
        xaxis=dict(title='Time'),
        yaxis=dict(title='Value of Change'),
        yaxis_range=[-50, 60],
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True, mode="immediate")]),
                     dict(label="Pause",
                          method="animate",
                          args=[[None], dict(frame=dict(duration=0, redraw=False), mode="immediate")])]
        )]
    ),
    frames=frames
)

# Configure the sliders
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
fig.update_layout(
    yaxis=dict(dtick=10),
    xaxis=dict(dtick=10),
    sliders=[{
        'steps': [{'args': [[f.name], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate'}],
                   'label': f'{f.name}',
                   'method': 'animate'} for f in frames],
        'transition': {'duration': 300},
    }]
)
fig.show()

## Exploit/Explore Model

In [137]:
class EEModel(Model):
    def __init__ (self, lo, hi, U, alpha, H):
        """
            q0(x, u), q1(x, u)
            c0(x, u), c1(x)
        """
        self.H = H
        self.n = 2 ** (H+1) - 1
        print(f'self.n: {self.n}')
        super().__init__(lo, hi, U, alpha)

    def construct_graph(self):
        """
            customize graph structure here
        """
        def add_children(idx):
            if self.nodes[idx].h == self.H:
                return
            up_idx = 2 * idx + 1
            down_idx = 2 * idx + 2
            self.draw_edge(idx, up_idx)
            self.draw_edge(idx, down_idx)
            self.nodes[up_idx].h = self.nodes[idx].h + 1
            self.nodes[down_idx].h = self.nodes[idx].h + 1
            self.nodes[up_idx].name = self.nodes[idx].name + "1"
            self.nodes[down_idx].name = self.nodes[idx].name + "0"
            add_children(up_idx)
            add_children(down_idx)

        self.root.h = 0
        self.root.name = "0"
        add_children(0)
    
    def construct_risks(self):
        """
            set Q, c, and one_step for each node
            u == 0: full exploit
            u == 1: full explore
        """
        
        def d_explore_exploit(s):
            last_zero_idx = -1
            last_one_idx = -1
            if '0' in s:
                last_zero_idx = s[::-1].index('0')
            if '1' in s:
                last_one_idx = s[::-1].index('1')
            
            return last_zero_idx, last_one_idx

        def q(name, x, u):
            """
                Takes in name of self, with x as state of parent, u as action on parent
            """
            d_exploit, d_explore = d_explore_exploit(name)
            is_explore = int(name[-1])
            if is_explore:
                # current node is exploring node
                if u == 0:
                    return {
                        0: 1
                    }
                if u == 5:
                    return {
                        d_explore: 0.2,
                        2 * d_explore: 0.4,
                        3 * d_explore: 0.4
                    }
                if u == 10:
                    return {
                        d_explore: 0.2,
                        2 * d_explore: 0.2,
                        6 * d_explore: 0.6,
                    }
            else:
                # current node is exploiting node
                if u == 0:
                    return {
                        x + max(0, 10 - d_exploit): 0.4,
                        x + max(0, 10 - 2 * d_exploit): 0.6
                    }
                if u == 5:
                    return {
                        x + 20: 0.4,
                        x + 20: 0.6,
                    }
                if u == 10:
                    return {
                        x + max(0, 10 - 6 * d_exploit): 0.4,
                        x + max(0, 10 - 8 * d_exploit): 0.6
                    }

        def c(x, u):
            # d_exploit, d_explore = d_explore_exploit(name)
            # is_explore = int(name[-1])
            if u == 0:
                return 0
            if u == 5:
                return 2.5
            if u == 10:
                return 5
        
        def c_term(x):
            return  3 * np.exp((1-x/self.hi))
        
        for node in self.nodes:
            node.isExplore = node.id % 2
            
            node.cost = {u : {x : c(x, u) for x in self.X} for u in self.U}                
            node.Q = {u : {x : self.bound(q(node.name, x, u)) for x in self.X} for u in self.U}

            if node.terminal:
                node.cost = {x : c_term(x) for x in self.X}
            #!customize one step here
            node.one_step = lambda list_qw : sum_of_AVAR(list_qw, self.alpha)

In [138]:
eemodel = EEModel(0, 50, [0, 5, 10], 0.4, 8)

self.n: 511


In [139]:
eemodel.root.calc_policy()

In [140]:
def turning_points(d):
    last0 = -1
    last5 = -1
    for k, v in d.items():
        if v == 0:
            last0 = k
        if v == 5:
            last5 = k
    return last0, last5

In [141]:
def print_turning_points(node, level = 0):
    print(" " * TAB_SIZE * level + f"{node.name}, {turning_points(node.policy)})")
    for child in node.children:
        if child.policy:
            print_turning_points(child, level+1)

print_turning_points(eemodel.root)

0, (50, -1))
    01, (50, -1))
        011, (50, -1))
            0111, (50, -1))
                01111, (50, -1))
                    011111, (50, -1))
                        0111111, (50, -1))
                            01111111, (50, -1))
                            01111110, (50, -1))
                        0111110, (50, -1))
                            01111101, (50, -1))
                            01111100, (50, -1))
                    011110, (50, -1))
                        0111101, (50, -1))
                            01111011, (50, -1))
                            01111010, (50, -1))
                        0111100, (50, -1))
                            01111001, (50, -1))
                            01111000, (50, -1))
                01110, (50, -1))
                    011101, (50, -1))
                        0111011, (50, -1))
                            01110111, (50, -1))
                            01110110, (50, -1))
                        0111010, (50, -1))


In [110]:
eemodel.nodes[133].cost[5][1]

2

## Disaster Relief Model

In [24]:
class DRModel(Model):
    def __init__ (self, lo, hi, U, alpha):
        """
            q0(x, u), q1(x, u)
            c0(x, u), c1(x)
        """
        super().__init__(lo, hi, U, alpha)

    def construct_graph(self):
        """
            customize graph structure here
        """
        adj_list = [(0, 1), (0, 2), (0, 3), (0, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 9), (2, 10), (2, 11), (3, 12), (3, 13), (4, 14), (4, 15)]
        for pa, ch in adj_list:
            self.draw_edge(pa, ch)
    
    def construct_risks(self):
        """
            set Q, c, and one_step for each node
        """
        def q(pa_x, pa_u, ch_idx):
            pct = pa_u[ch_idx]
            val = floor(pa_x * pct /100)
            return {
                val: 0.9,
                floor(val * 0.9): 0.1
            }
        
        def c(x, u):
            return x * max(u)

        def c_term(x, necessary):
            loss = max(necessary - x, 0)
            return np.exp(loss)
        
        universal_cost = {u : {x: c(x, u) for x in self.X} for u in self.U}
    
        for node in self.nodes:
            print(f'at node {node.id}')
            if not node.terminal:
                node.cost = universal_cost
                node.one_step = lambda list_qw : sum_of_AVAR(list_qw, self.alpha)
                for ch_idx, child in enumerate(node.children):
                    print(f'setting for {child.id}')
                    child.Q = {u: {x : self.bound(q(x, u, ch_idx)) for x in self.X} for u in self.U} 
                    print(f'done setting for {child.id}')
            else:
                node.cost = {x : c_term(x, node.id) for x in self.X}
    # customized functions for this particular model

In [25]:
import itertools
MAX_U = 20
MAX_CH = 4
tuples = list(itertools.combinations_with_replacement(range(MAX_U + 1), MAX_CH))
U = [t for t in tuples if sum(t) == MAX_U]
print(len(U))

108


In [26]:
drmodel = DRModel(0, 200, U, 0.3)

at node 0
setting for 1
done setting for 1
setting for 2
done setting for 2
setting for 3
done setting for 3
setting for 4
done setting for 4
at node 1
setting for 5
done setting for 5
setting for 6
done setting for 6
setting for 7
done setting for 7
setting for 8
done setting for 8
at node 2
setting for 9
done setting for 9
setting for 10
done setting for 10
setting for 11
done setting for 11
at node 3
setting for 12
done setting for 12
setting for 13
done setting for 13
at node 4
setting for 14
done setting for 14
setting for 15
done setting for 15
at node 5
at node 6
at node 7
at node 8
at node 9
at node 10
at node 11
at node 12
at node 13
at node 14
at node 15


In [27]:
drmodel.root.calc_policy()

In [ ]:
import pickle
import time
current_time = time.strftime("%m%d-%H%M%S")
directory = f"pickles/{current_time}"
for node in drmodel.nodes:
    if node.policy:
        policy_pickle = f"{directory}/policy_{node.id}.pickle"
        with open(policy_pickle, "wb") as f:
            pickle.dump(node.policy, f)
    
    if node.w:
        w_pickle = f"{directory}/w_{node.id}.pickle"dd
        with open(w_pickle, "wb") as f:
            pickle.dump(node.w, f)

Policy is empty for node 5
Policy is empty for node 6
Policy is empty for node 7
Policy is empty for node 8
Policy is empty for node 9
Policy is empty for node 10
Policy is empty for node 11
Policy is empty for node 12
Policy is empty for node 13
Policy is empty for node 14
Policy is empty for node 15
